# FULL PIPELINE (COLAB) — HOLDOUT + KFOLD + TOKENIZATION CHECK

In [ ]:
# ============================================================
# Models: IndoBERT, IndoBERTweet, IndoRoBERTa
# Regularization: Dropout (head) + L2 (AdamW weight_decay) + EarlyStopping
# Save models to:
# /content/drive/MyDrive/SKRIPSI/models/<MODEL_TAG>/
#   ├── holdout/<SCHEME>/
#   └── kfold/k<K>/fold<F>/
# ============================================================

# ---------- 0) Imports ----------
import os, time, json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import (
    AutoTokenizer, AutoModel, AutoConfig,
    Trainer, TrainingArguments, EarlyStoppingCallback
)
from transformers.modeling_outputs import SequenceClassifierOutput

In [1]:
# ===== 1) Mount Google Drive =====
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ---------- 2) Paths ----------
SPLIT_HOLDOUT_ROOT = "/content/drive/MyDrive/SKRIPSI/split/HO"
SPLIT_KFOLD_ROOT   = "/content/drive/MyDrive/SKRIPSI/split/K"
MODELS_ROOT        = "/content/drive/MyDrive/SKRIPSI/models"
RESULTS_ROOT       = "/content/drive/MyDrive/SKRIPSI/results"

os.makedirs(MODELS_ROOT, exist_ok=True)
os.makedirs(RESULTS_ROOT, exist_ok=True)

In [ ]:
# ---------- 3) Dataset columns ----------
TEXT_COL  = "text"
LABEL_COL = "label_id"
NUM_LABELS = 6

In [ ]:
# ---------- 4) Model Zoo ----------
MODEL_ZOO = {
    "IndoBERT":     "indobenchmark/indobert-base-p1",
    "IndoBERTweet": "indolem/indobertweet-base-uncased",
    "IndoRoBERTa":  "cahya/roberta-base-indonesian-522M",
}

## Check PATH Datasets

In [ ]:
import os

HO_ROOT = "/content/drive/MyDrive/SKRIPSI/split/HO"
folder_ho = ["HO_80_10_10", "HO_70_15_15", "HO_90_05_05"]
schemes   = ["80_10_10", "70_15_15", "90_05_05"]   # nama file

print("===== CHECK HOLDOUT FILES (folder HO_xxx + file train_xxx.csv) =====")
print("HO root exists ->", os.path.exists(HO_ROOT))

for folder_name, scheme in zip(folder_ho, schemes):
    sc_dir = os.path.join(HO_ROOT, folder_name)

    print(f"\nChecking folder: {folder_name} | files: *_{scheme}.csv")
    print(" folder ->", os.path.exists(sc_dir))

    train_path = os.path.join(sc_dir, f"train_{scheme}.csv")
    val_path   = os.path.join(sc_dir, f"val_{scheme}.csv")
    test_path  = os.path.join(sc_dir, f"test_{scheme}.csv")

    print(" train ->", os.path.exists(train_path))
    print(" val   ->", os.path.exists(val_path))
    print(" test  ->", os.path.exists(test_path))

===== CHECK HOLDOUT FILES (folder HO_xxx + file train_xxx.csv) =====
HO root exists -> True

Checking folder: HO_80_10_10 | files: *_80_10_10.csv
 folder -> True
 train -> True
 val   -> True
 test  -> True

Checking folder: HO_70_15_15 | files: *_70_15_15.csv
 folder -> True
 train -> True
 val   -> True
 test  -> True

Checking folder: HO_90_05_05 | files: *_90_05_05.csv
 folder -> True
 train -> True
 val   -> True
 test  -> True


In [ ]:
SPLIT_KFOLD_ROOT = "/content/drive/MyDrive/SKRIPSI/split/K"

print("\n===== CHECK KFOLD FILES =====")

for k in range(2, 11):
    k_dir = os.path.join(SPLIT_KFOLD_ROOT, f"k{k}")

    print(f"\nChecking k{k}")
    print("folder exists ->", os.path.exists(k_dir))

    if os.path.exists(k_dir):
        for fold in range(1, k + 1):
            train_path = os.path.join(k_dir, f"train_fold{fold}.csv")
            val_path   = os.path.join(k_dir, f"val_fold{fold}.csv")

            print(f" fold{fold} train ->", os.path.exists(train_path))
            print(f" fold{fold} val   ->", os.path.exists(val_path))


===== CHECK KFOLD FILES =====

Checking k2
folder exists -> True
 fold1 train -> True
 fold1 val   -> True
 fold2 train -> True
 fold2 val   -> True

Checking k3
folder exists -> True
 fold1 train -> True
 fold1 val   -> True
 fold2 train -> True
 fold2 val   -> True
 fold3 train -> True
 fold3 val   -> True

Checking k4
folder exists -> True
 fold1 train -> True
 fold1 val   -> True
 fold2 train -> True
 fold2 val   -> True
 fold3 train -> True
 fold3 val   -> True
 fold4 train -> True
 fold4 val   -> True

Checking k5
folder exists -> True
 fold1 train -> True
 fold1 val   -> True
 fold2 train -> True
 fold2 val   -> True
 fold3 train -> True
 fold3 val   -> True
 fold4 train -> True
 fold4 val   -> True
 fold5 train -> True
 fold5 val   -> True

Checking k6
folder exists -> True
 fold1 train -> True
 fold1 val   -> True
 fold2 train -> True
 fold2 val   -> True
 fold3 train -> True
 fold3 val   -> True
 fold4 train -> True
 fold4 val   -> True
 fold5 train -> True
 fold5 val   -> T

## Define Function

In [ ]:
# ===== 5) Hyperparameters  =====
MAX_LEN = 128
LR = 2e-5
BATCH_SIZE = 16
EPOCHS = 20                 # maksimum; early stopping bisa stop lebih cepat
DROPOUT_RATE = 0.3
WEIGHT_DECAY = 0.01         # L2/weight decay
EARLY_PATIENCE = 2
EARLY_THRESHOLD = 0.0
SEED = 42


In [ ]:
# ---------- 6) Dataset class ----------
class TextClsDataset(torch.utils.data.Dataset):
    def __init__(self, df: pd.DataFrame, tokenizer, max_len: int = 128):
        self.texts = df[TEXT_COL].astype(str).tolist()
        self.labels = df[LABEL_COL].astype(int).tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self): return len(self.texts)

    def __getitem__(self, idx: int):
        enc = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        item = {k: v.squeeze(0) for k, v in enc.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

In [ ]:
# ---------- 7) Custom model: Dropout head ----------
class TransformerWithDropoutHead(nn.Module):
    def __init__(self, model_name: str, num_labels: int = 6, dropout_rate: float = 0.3):
        super().__init__()
        config = AutoConfig.from_pretrained(model_name)
        # internal dropout (kalau didukung)
        setattr(config, "hidden_dropout_prob", dropout_rate)
        setattr(config, "attention_probs_dropout_prob", dropout_rate)

        self.backbone = AutoModel.from_pretrained(model_name, config=config)
        hidden_size = self.backbone.config.hidden_size

        self.dropout = nn.Dropout(dropout_rate)  # dropout tambahan di head
        self.classifier = nn.Linear(hidden_size, num_labels)
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, **kwargs):
        forward_args = {"input_ids": input_ids, "attention_mask": attention_mask}
        # RoBERTa-like biasanya tidak pakai token_type_ids
        if "token_type_ids" in self.backbone.forward.__code__.co_varnames and token_type_ids is not None:
            forward_args["token_type_ids"] = token_type_ids

        outputs = self.backbone(**forward_args)
        cls = outputs.last_hidden_state[:, 0, :]
        x = self.dropout(cls)
        logits = self.classifier(x)

        loss = None
        if labels is not None:
            loss = self.loss_fn(logits, labels)

        return SequenceClassifierOutput(loss=loss, logits=logits)

In [ ]:
# ---------- 8) Metrics ----------
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, preds)
    prec, rec, f1, _ = precision_recall_fscore_support(labels, preds, average="macro", zero_division=0)
    return {"accuracy": acc, "precision_macro": prec, "recall_macro": rec, "f1_macro": f1}

In [ ]:
# ---------- 9) Tokenization sanity check ----------
def check_tokenization_for_all_models(sample_text: str, max_len: int = 128):
    """
    Cek tokenisasi untuk 3 model:
    - input_ids shape
    - attention_mask shape
    - forward pass backbone (opsional tapi bagus)
    """
    print("\n================ TOKENIZATION CHECK ================")
    print("Sample text:", sample_text[:200], "..." if len(sample_text) > 200 else "")
    for tag, model_name in MODEL_ZOO.items():
        print(f"\n--- {tag} ({model_name}) ---")
        tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)
        enc = tok(
            sample_text,
            truncation=True,
            padding="max_length",
            max_length=max_len,
            return_tensors="pt"
        )
        print("input_ids:", tuple(enc["input_ids"].shape))
        print("attention_mask:", tuple(enc["attention_mask"].shape))

        # forward pass cepat (tanpa grad)
        backbone = AutoModel.from_pretrained(model_name)
        backbone.eval()
        with torch.no_grad():
            out = backbone(**enc)
        print("last_hidden_state:", tuple(out.last_hidden_state.shape))
    print("====================================================\n")

In [ ]:
# ---------- 10) Train 1 run (model + 1 split) ----------
def train_one_run(
    model_tag: str,
    hf_model_name: str,
    train_df: pd.DataFrame,
    val_df: pd.DataFrame,
    out_dir: str,
):
    os.makedirs(out_dir, exist_ok=True)

    tokenizer = AutoTokenizer.from_pretrained(hf_model_name, use_fast=True)
    train_ds = TextClsDataset(train_df, tokenizer, max_len=MAX_LEN)
    val_ds   = TextClsDataset(val_df, tokenizer, max_len=MAX_LEN)

    model = TransformerWithDropoutHead(hf_model_name, num_labels=NUM_LABELS, dropout_rate=DROPOUT_RATE)

    args = TrainingArguments(
        output_dir=out_dir,
        run_name=os.path.basename(out_dir),
        learning_rate=LR,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        num_train_epochs=EPOCHS,
        weight_decay=WEIGHT_DECAY,
        eval_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="f1_macro",
        greater_is_better=True,
        logging_strategy="epoch",
        fp16=torch.cuda.is_available(),
        report_to="none",
        seed=SEED
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(
            early_stopping_patience=EARLY_PATIENCE,
            early_stopping_threshold=EARLY_THRESHOLD
        )]
    )

    t0 = time.time()
    train_result = trainer.train()
    t1 = time.time()
    eval_result = trainer.evaluate()

    # ===== SIMPAN LOG TRAINING =====
    log_history = trainer.state.log_history
    pd.DataFrame(log_history).to_csv(
      os.path.join(out_dir, "log_history.csv"),
      index=False
    )

    # ===== SIMPAN PREDIKSI VALIDATION =====
    pred_output = trainer.predict(val_ds)

    val_preds = np.argmax(pred_output.predictions, axis=1)
    val_labels = pred_output.label_ids

    pd.DataFrame({
        "true_label": val_labels,
        "pred_label": val_preds
    }).to_csv(
        os.path.join(out_dir, "val_predictions.csv"),
        index=False
    )

    trainer.save_model(out_dir)
    tokenizer.save_pretrained(out_dir)

    summary = {
        "model_tag": model_tag,
        "hf_model_name": hf_model_name,
        "out_dir": out_dir,
        "train_time_sec": round(t1 - t0, 2),
        "train_metrics": {k: float(v) for k, v in train_result.metrics.items()},
        "eval_metrics": {k: float(v) for k, v in eval_result.items()},
        "hyperparams": {
            "max_len": MAX_LEN,
            "lr": LR,
            "batch_size": BATCH_SIZE,
            "epochs_max": EPOCHS,
            "dropout_rate": DROPOUT_RATE,
            "weight_decay": WEIGHT_DECAY,
            "early_patience": EARLY_PATIENCE,
            "early_threshold": EARLY_THRESHOLD
        }
    }
    with open(os.path.join(out_dir, "summary.json"), "w", encoding="utf-8") as f:
        json.dump(summary, f, ensure_ascii=False, indent=2)

    return summary

# Hold Out Schemes

In [ ]:
def run_holdout_training():
    HO_ROOT = "/content/drive/MyDrive/SKRIPSI/split/HO"

    folder_ho = ["HO_80_10_10", "HO_70_15_15", "HO_90_05_05"]
    schemes   = ["80_10_10", "70_15_15", "90_05_05"]

    rows = []

    for folder_name, scheme in zip(folder_ho, schemes):
        sc_dir = os.path.join(HO_ROOT, folder_name)

        train_path = os.path.join(sc_dir, f"train_{scheme}.csv")
        val_path   = os.path.join(sc_dir, f"val_{scheme}.csv")

        if not (os.path.exists(train_path) and os.path.exists(val_path)):
            print(f"[HOLDOUT SKIP] File tidak lengkap untuk {folder_name}")
            continue

        train_df = pd.read_csv(train_path)
        val_df   = pd.read_csv(val_path)

        # cek kolom
        if TEXT_COL not in train_df.columns or LABEL_COL not in train_df.columns:
            raise ValueError(f"Kolom {TEXT_COL}/{LABEL_COL} tidak ada di {train_path}")
        if TEXT_COL not in val_df.columns or LABEL_COL not in val_df.columns:
            raise ValueError(f"Kolom {TEXT_COL}/{LABEL_COL} tidak ada di {val_path}")

        print(f"\n===== HOLDOUT TRAINING: {folder_name} =====")

        # 🔥 train 3 model
        for model_tag, hf_name in MODEL_ZOO.items():

            out_dir = os.path.join(
                MODELS_ROOT,
                model_tag,
                "holdout",
                folder_name   # pakai nama folder biar rapi
            )

            print(f"--- Training {model_tag} ---")

            summary = train_one_run(
                model_tag,
                hf_name,
                train_df,
                val_df,
                out_dir
            )

            ev = summary["eval_metrics"]

            rows.append({
                "split_type": "holdout",
                "scheme": folder_name,
                "model": model_tag,
                "train_time_sec": summary["train_time_sec"],
                "eval_loss": ev.get("eval_loss"),
                "eval_accuracy": ev.get("eval_accuracy"),
                "eval_precision_macro": ev.get("eval_precision_macro"),
                "eval_recall_macro": ev.get("eval_recall_macro"),
                "eval_f1_macro": ev.get("eval_f1_macro"),
                "saved_to": summary["out_dir"]
            })

    out_csv = os.path.join(RESULTS_ROOT, "results_holdout_final.csv")
    pd.DataFrame(rows).to_csv(out_csv, index=False)

    print("\n[SUKSES] Holdout selesai.")
    print("Rekap:", out_csv)

    return pd.DataFrame(rows)

# K-FOLD Schemes

In [ ]:
# ---------- 12) K-FOLD TRAINING ----------
def run_kfold_training(k_values=range(2, 11), token_check_fold=1):
    """
    Expect folder:
      /split/K/k2/train_fold1.csv, val_fold1.csv, ..., train_fold2.csv...
    Tokenization check:
      - per k (default hanya fold 1 biar tidak berat) => token_check_fold=1
    Save:
      /models/<ModelTag>/kfold/k2/fold1/
    """
    rows = []

    for k in k_values:
        k_dir = os.path.join(SPLIT_KFOLD_ROOT, f"k{k}")
        if not os.path.isdir(k_dir):
            print(f"[KFOLD SKIP] Folder tidak ditemukan: {k_dir}")
            continue

        train_files = sorted([f for f in os.listdir(k_dir) if f.startswith("train_fold") and f.endswith(".csv")])
        num_folds = len(train_files)
        if num_folds == 0:
            print(f"[KFOLD SKIP] Tidak ada train_fold*.csv di {k_dir}")
            continue

        # ✅ tokenisasi check per k (ambil fold tertentu saja biar ringan)
        chk_fold = token_check_fold
        chk_train_path = os.path.join(k_dir, f"train_fold{chk_fold}.csv")
        if os.path.exists(chk_train_path):
            chk_train_df = pd.read_csv(chk_train_path)
            if TEXT_COL not in chk_train_df.columns:
                raise ValueError(f"Kolom {TEXT_COL} tidak ada di {chk_train_path}")
            sample_text = chk_train_df[TEXT_COL].iloc[0]
            print(f"\n### KFOLD k={k} — TOKENIZATION CHECK (fold {chk_fold}) ###")
            check_tokenization_for_all_models(sample_text, max_len=MAX_LEN)
        else:
            print(f"[KFOLD WARNING] Tidak bisa token check k={k}, fold {chk_fold} (file tidak ada)")

        # training semua fold untuk k itu
        for fold in range(1, num_folds + 1):
            train_path = os.path.join(k_dir, f"train_fold{fold}.csv")
            val_path   = os.path.join(k_dir, f"val_fold{fold}.csv")

            train_df = pd.read_csv(train_path)
            val_df   = pd.read_csv(val_path)

            if TEXT_COL not in train_df.columns or LABEL_COL not in train_df.columns:
                raise ValueError(f"Kolom {TEXT_COL}/{LABEL_COL} tidak ada di {train_path}")
            if TEXT_COL not in val_df.columns or LABEL_COL not in val_df.columns:
                raise ValueError(f"Kolom {TEXT_COL}/{LABEL_COL} tidak ada di {val_path}")

            for model_tag, hf_name in MODEL_ZOO.items():
                # simpan: /models/<ModelTag>/kfold/k{k}/fold{fold}/
                out_dir = os.path.join(MODELS_ROOT, model_tag, "kfold", f"k{k}", f"fold{fold}")
                print(f"\n===== TRAIN KFOLD k={k} fold={fold} | {model_tag} =====")
                try:
                    summary = train_one_run(model_tag, hf_name, train_df, val_df, out_dir)

                    ev = summary["eval_metrics"]
                    rows.append({
                        "split_type": "kfold",
                        "scheme": f"k{k}",
                        "fold": fold,
                        "model": model_tag,
                        "status": "OK",
                        "error": "",
                        "train_time_sec": summary.get("train_time_sec"),
                        "eval_loss": ev.get("eval_loss"),
                        "eval_accuracy": ev.get("eval_accuracy"),
                        "eval_precision_macro": ev.get("eval_precision_macro"),
                        "eval_recall_macro": ev.get("eval_recall_macro"),
                        "eval_f1_macro": ev.get("eval_f1_macro"),
                        "saved_to": summary.get("out_dir", out_dir),
                    })

                except Exception as e:
                    rows.append({
                        "split_type": "kfold",
                        "scheme": f"k{k}",
                        "fold": fold,
                        "model": model_tag,
                        "status": "FAILED",
                        "error": repr(e),
                        "train_time_sec": None,
                        "eval_loss": None,
                        "eval_accuracy": None,
                        "eval_precision_macro": None,
                        "eval_recall_macro": None,
                        "eval_f1_macro": None,
                        "saved_to": out_dir,
                    })
                    print(f"[ERROR] k={k} fold={fold} model={model_tag} -> {repr(e)}")

                pd.DataFrame(rows).to_csv(
                    os.path.join(RESULTS_ROOT, "results_kfold_progress.csv"),
                    index=False
                )
    out_csv = os.path.join(RESULTS_ROOT, "results_kfold_final.csv")
    pd.DataFrame(rows).to_csv(out_csv, index=False)
    print("\n[SUKSES] K-Fold selesai. Rekap:", out_csv)
    return pd.DataFrame(rows)

# Mini Test for Checking

In [ ]:
def tokenization_sanity_check_once(sample_text: str, max_len: int = 128):
    """
    Cek tokenisasi 1x untuk 3 model (sekali sebelum semua training).
    """
    from transformers.utils import logging as hf_logging
    hf_logging.set_verbosity_error()   # sembunyikan warning/info dari transformers
    print("\n========== TOKENIZATION SANITY CHECK (ONCE) ==========")
    print("Sample text:", sample_text[:200], "..." if len(sample_text) > 200 else "")

    for tag, model_name in MODEL_ZOO.items():
        print(f"\n--- {tag} ({model_name}) ---")
        tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)
        enc = tok(
            sample_text,
            truncation=True,
            padding="max_length",
            max_length=max_len,
            return_tensors="pt"
        )
        print(" input_ids:", tuple(enc["input_ids"].shape))
        print(" attention_mask:", tuple(enc["attention_mask"].shape))

        # OPTIONAL: forward pass cepat (biar yakin input cocok)
        backbone = AutoModel.from_pretrained(model_name)
        backbone.eval()
        with torch.no_grad():
            out = backbone(**enc)
        print(" last_hidden_state:", tuple(out.last_hidden_state.shape))

    print("======================================================\n")

In [ ]:
# ========= Ambil sample text dari salah satu split (holdout kalau ada) =========
sample_text = None

# coba ambil dari holdout 80_10_10
holdout_sample_path = os.path.join(SPLIT_HOLDOUT_ROOT, "train_80_10_10.csv")
if os.path.exists(holdout_sample_path):
    df_tmp = pd.read_csv(holdout_sample_path)
    sample_text = df_tmp[TEXT_COL].astype(str).iloc[0]
else:
    # kalau holdout ga ada, ambil dari kfold k2 fold1
    kfold_sample_path = os.path.join(SPLIT_KFOLD_ROOT, "k2", "train_fold1.csv")
    if os.path.exists(kfold_sample_path):
        df_tmp = pd.read_csv(kfold_sample_path)
        sample_text = df_tmp[TEXT_COL].astype(str).iloc[0]

if sample_text is None:
    raise FileNotFoundError("Tidak menemukan file sample untuk token check (holdout train_80_10_10 atau kfold k2/train_fold1).")

# ========= TOKEN CHECK SEKALI AJA =========
tokenization_sanity_check_once(sample_text, max_len=MAX_LEN)


========== TOKENIZATION SANITY CHECK (ONCE) ==========
Sample text: pagi udah di buat emosi 

--- IndoBERT (indobenchmark/indobert-base-p1) ---
 input_ids: (1, 128)
 attention_mask: (1, 128)


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

 last_hidden_state: (1, 128, 768)

--- IndoBERTweet (indolem/indobertweet-base-uncased) ---
 input_ids: (1, 128)
 attention_mask: (1, 128)


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

 last_hidden_state: (1, 128, 768)

--- IndoRoBERTa (cahya/roberta-base-indonesian-522M) ---
 input_ids: (1, 128)
 attention_mask: (1, 128)


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

 last_hidden_state: (1, 128, 768)



# Run All

In [ ]:
# ============================================================
# 13) RUN EVERYTHING
# ============================================================

# (A) Hold-out
holdout_df = run_holdout_training()

# (B) K-fold (k=2..10)
kfold_df = run_kfold_training(k_values=range(2, 11), token_check_fold=1)

# (C) Gabungkan rekap (opsional)
all_df = pd.concat([holdout_df, kfold_df], ignore_index=True)
all_path = os.path.join(RESULTS_ROOT, "results_all_final.csv")
all_df.to_csv(all_path, index=False)

print("\n[SUKSES] Semua selesai.")
print("Model tersimpan di:", MODELS_ROOT)
print("Rekap gabungan:", all_path)


===== HOLDOUT TRAINING: HO_80_10_10 =====
--- Training IndoBERT ---


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

{'loss': '0.9824', 'grad_norm': '36.93', 'learning_rate': '1.9e-05', 'epoch': '1'}
{'eval_loss': '0.6781', 'eval_accuracy': '0.7553', 'eval_precision_macro': '0.7542', 'eval_recall_macro': '0.7885', 'eval_f1_macro': '0.7631', 'eval_runtime': '0.8552', 'eval_samples_per_second': '822', 'eval_steps_per_second': '51.45', 'epoch': '1'}
{'loss': '0.657', 'grad_norm': '28.56', 'learning_rate': '1.8e-05', 'epoch': '2'}
{'eval_loss': '0.6819', 'eval_accuracy': '0.7738', 'eval_precision_macro': '0.7743', 'eval_recall_macro': '0.7946', 'eval_f1_macro': '0.7808', 'eval_runtime': '0.8832', 'eval_samples_per_second': '796', 'eval_steps_per_second': '49.82', 'epoch': '2'}
{'loss': '0.558', 'grad_norm': '5.55', 'learning_rate': '1.7e-05', 'epoch': '3'}
{'eval_loss': '0.8127', 'eval_accuracy': '0.7525', 'eval_precision_macro': '0.7535', 'eval_recall_macro': '0.7898', 'eval_f1_macro': '0.7598', 'eval_runtime': '0.8526', 'eval_samples_per_second': '824.5', 'eval_steps_per_second': '51.6', 'epoch': '3'}


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

{'loss': '1.623', 'grad_norm': '81.73', 'learning_rate': '1.9e-05', 'epoch': '1'}
{'eval_loss': '0.9648', 'eval_accuracy': '0.67', 'eval_precision_macro': '0.684', 'eval_recall_macro': '0.6719', 'eval_f1_macro': '0.6667', 'eval_runtime': '0.9235', 'eval_samples_per_second': '761.3', 'eval_steps_per_second': '47.65', 'epoch': '1'}
{'loss': '0.9606', 'grad_norm': '131.9', 'learning_rate': '1.8e-05', 'epoch': '2'}
{'eval_loss': '0.7488', 'eval_accuracy': '0.7553', 'eval_precision_macro': '0.7599', 'eval_recall_macro': '0.7824', 'eval_f1_macro': '0.7661', 'eval_runtime': '0.8452', 'eval_samples_per_second': '831.8', 'eval_steps_per_second': '52.06', 'epoch': '2'}
{'loss': '0.7776', 'grad_norm': '39.16', 'learning_rate': '1.7e-05', 'epoch': '3'}
{'eval_loss': '0.7562', 'eval_accuracy': '0.7639', 'eval_precision_macro': '0.7616', 'eval_recall_macro': '0.7995', 'eval_f1_macro': '0.7729', 'eval_runtime': '0.8969', 'eval_samples_per_second': '783.8', 'eval_steps_per_second': '49.06', 'epoch': '

...

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

{'loss': '1.257', 'grad_norm': '26.67', 'learning_rate': '1.9e-05', 'epoch': '1'}
{'eval_loss': '0.9984', 'eval_accuracy': '0.6743', 'eval_precision_macro': '0.6719', 'eval_recall_macro': '0.6967', 'eval_f1_macro': '0.6812', 'eval_runtime': '1.1', 'eval_samples_per_second': '798.5', 'eval_steps_per_second': '50.02', 'epoch': '1'}
{'loss': '0.8961', 'grad_norm': '10.81', 'learning_rate': '1.8e-05', 'epoch': '2'}
{'eval_loss': '1.029', 'eval_accuracy': '0.6868', 'eval_precision_macro': '0.6851', 'eval_recall_macro': '0.7059', 'eval_f1_macro': '0.6921', 'eval_runtime': '1.158', 'eval_samples_per_second': '758.4', 'eval_steps_per_second': '47.51', 'epoch': '2'}
{'loss': '0.7705', 'grad_norm': '8.491', 'learning_rate': '1.7e-05', 'epoch': '3'}
{'eval_loss': '0.9304', 'eval_accuracy': '0.6879', 'eval_precision_macro': '0.6919', 'eval_recall_macro': '0.7011', 'eval_f1_macro': '0.6926', 'eval_runtime': '1.116', 'eval_samples_per_second': '786.5', 'eval_steps_per_second': '49.27', 'epoch': '3'}

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

{'loss': '0.9745', 'grad_norm': '20.56', 'learning_rate': '1.9e-05', 'epoch': '1'}
{'eval_loss': '0.8036', 'eval_accuracy': '0.7403', 'eval_precision_macro': '0.7417', 'eval_recall_macro': '0.7707', 'eval_f1_macro': '0.7491', 'eval_runtime': '1.141', 'eval_samples_per_second': '769.5', 'eval_steps_per_second': '48.2', 'epoch': '1'}


RuntimeError: [enforce fail at inline_container.cc:664] . unexpected pos 329289728 vs 329289620